# Parking Lot OOP

## Goals: Design a parking lot using object-oriented principles

Here are a few methods that you should be able to run:

- Tell us how many spots are remaining
- Tell us how many total spots are in the parking lot
- Tell us when the parking lot is full
- Tell us when the parking lot is empty
- Tell us when certain spots are full e.g. when all motorcycle spots are taken
- Tell us how many spots vans are taking up

## Assumptions:

- The parking lot can hold motorcycles, cars and vans
- The parking lot has motorcycle spots, car spots and large spots
- A motorcycle can park in any spot
- A car can park in a regular or large spot
- A van can park in a large spot, but it will take up 3 regular spots

In [1]:
from enum import Enum

class VehicleType(Enum):
    CAR = 1
    BIKE = 2
    TRUCK = 3
    
class SpotType(Enum):
    SMALL = 1
    MEDIUM = 2
    LARGE = 3
    
class Vehicle:
    def __init__(self,type:VehicleType):
        self.type = type
        
    def __str__(self):
        return f"Vehicle of type {self.type}"
    
    def get_type(self):
        return self.type
    
class ParkingSpot:
    def __init__(self,type:SpotType):
        self.type = type
        self.vehicle = None
        
    def is_empty(self):
        return self.vehicle is None
    
    def remove_vehicle(self):
        self.vehicle = None
        
    def get_vehicle(self):
        return self.vehicle.get_type() if self.vehicle else None
    
    def park(self,vehicle:Vehicle):
        self.vehicle = vehicle
        
class ParkingLot:
    def __init__(self,small_spots:int,medium_spots:int,large_spots:int):
        self.small_spots = small_spots
        self.medium_spots = medium_spots
        self.large_spots = large_spots
        self.spots = {
            SpotType.SMALL: [ParkingSpot(SpotType.SMALL) for _ in range(small_spots)],
            SpotType.MEDIUM: [ParkingSpot(SpotType.MEDIUM) for _ in range(medium_spots)],
            SpotType.LARGE: [ParkingSpot(SpotType.LARGE) for _ in range(large_spots)]
        }
        self.total_spots = small_spots + medium_spots+ large_spots
        
    def is_empty(self):
        return self.get_total_spots() == self.get_total_available_spots()
    
    def is_full(self):
        return self.get_total_spots() == self.get_total_parked_spots()
            
    def get_total_spots(self):
        return self.total_spots
    
    def get_total_available_spots(self):
        return sum(spot.is_empty() for spots in self.spots.values() for spot in spots)
    
    def get_total_parked_spots(self):
        return sum(not spot.is_empty() for spots in self.spots.values() for spot in spots)
    
    def get_available_spots(self,spot_type:SpotType):
        return len([spot for spot in self.spots[spot_type] if spot.is_empty()])
    
    def park_vehicle(self,vehicle:Vehicle):
        try:
            if vehicle.get_type() == VehicleType.CAR:
                return self.park_car()
            elif vehicle.get_type() == VehicleType.BIKE:
                return self.park_bike()
            elif vehicle.get_type() == VehicleType.TRUCK:
                return self.park_truck()
            else:
                raise Exception("Invalid vehicle type")
        except Exception as e:
            print(e)
            return False
        
    def park_bike(self):
        if self.get_total_available_spots() > 0:
            for spot_type in [SpotType.SMALL,SpotType.MEDIUM,SpotType.LARGE]:
                for spot in self.spots[spot_type]:
                    if spot.is_empty():
                        spot.park(Vehicle(VehicleType.BIKE))
                        print(f"Parked bike at spot {spot.type}")
                        return True
        return False
    
    def park_car(self):
        if self.get_available_spots(SpotType.MEDIUM) > 0 or self.get_available_spots(SpotType.LARGE) > 0:
            for spot_type in [SpotType.MEDIUM,SpotType.LARGE]:
                for spot in self.spots[spot_type]:
                    if spot.is_empty():
                        spot.park(Vehicle(VehicleType.CAR))
                        print(f"Parked car at spot {spot.type}")    
                        return True
        return False
    
    def park_truck(self):
        if self.get_available_spots(SpotType.LARGE) > 0:
            for spot_type in [SpotType.LARGE]:
                for spot in self.spots[spot_type]:
                    if spot.is_empty():
                        spot.park(Vehicle(VehicleType.TRUCK))
                        print(f"Parked truck at spot {spot.type}")
                        return True
        elif self.get_available_spots(SpotType.MEDIUM) >= 3:
            available_medium_spots = [spot for spot in self.spots[SpotType.MEDIUM] if spot.is_empty()]
            for i in range(0,len(available_medium_spots),3):
                spots = available_medium_spots[i:i+3]
                if all([spot.is_empty() for spot in spots]):
                    for spot in spots:
                        spot.park(Vehicle(VehicleType.TRUCK))
                    print(f"Parked truck at spot {spot.type}")
                    return True
        return False
    
    def parked_bike_spots(self):
        bike_spots = []
        for spot_type in [SpotType.SMALL,SpotType.MEDIUM,SpotType.LARGE]:
            for spot in self.spots[spot_type]:
                if not spot.is_empty() and spot.get_vehicle() == VehicleType.BIKE:
                    bike_spots.append(spot)
        return len(bike_spots)
    
    def parked_car_spots(self):
        car_spots = []
        for spot_type in [SpotType.MEDIUM,SpotType.LARGE]:
            for spot in self.spots[spot_type]:
                if not spot.is_empty() and spot.get_vehicle() == VehicleType.CAR:
                    car_spots.append(spot)
        return len(car_spots)
    
    def parked_truck_spots(self):
        large_spots = [spot for spot in self.spots[SpotType.LARGE] if not spot.is_empty() and spot.get_vehicle() == VehicleType.TRUCK]
        medium_spots = [spot for spot in self.spots[SpotType.MEDIUM] if not spot.is_empty() and spot.get_vehicle() == VehicleType.TRUCK]
        # truck_spots = large_spots + medium_spots
        return len(large_spots) + len(medium_spots)//3
   
 


In [2]:
# example
parking_lot = ParkingLot(small_spots=10, medium_spots=20, large_spots=5)

print(f"Total spots: {parking_lot.get_total_spots()}")
print(f"Remaining spots: {parking_lot.get_total_available_spots()}")
print(f"Is parking lot full? {parking_lot.is_full()}")
print(f"Is parking lot empty? {parking_lot.is_empty()}")
print(f"Available motorcycle spots: {parking_lot.get_available_spots(SpotType.SMALL)}")
print(f"Available car spots: {parking_lot.get_available_spots(SpotType.MEDIUM)}")
print(f"Available truck spots: {parking_lot.get_available_spots(SpotType.LARGE)}")
print(f"Parked bike spots: {parking_lot.parked_bike_spots()}")
print(f"Parked car spots: {parking_lot.parked_car_spots()}")
print(f"Parked truck spots: {parking_lot.parked_truck_spots()}")

# parking some vehicles
n_bikes = 0
n_cars = 0
n_trucks = 12
for _ in range(n_bikes):
    status = parking_lot.park_vehicle(Vehicle(VehicleType.BIKE))
for _ in range(n_cars):
    status =parking_lot.park_vehicle(Vehicle(VehicleType.CAR))
for _ in range(n_trucks):
    status =parking_lot.park_vehicle(Vehicle(VehicleType.TRUCK))
    

print(f"Total spots: {parking_lot.get_total_spots()}")
print(f"Remaining spots: {parking_lot.get_total_available_spots()}")
print(f"Is parking lot full? {parking_lot.is_full()}")
print(f"Is parking lot empty? {parking_lot.is_empty()}")
print(f"Available small spots: {parking_lot.get_available_spots(SpotType.SMALL)}")
print(f"Available medium spots: {parking_lot.get_available_spots(SpotType.MEDIUM)}")
print(f"Available large spots: {parking_lot.get_available_spots(SpotType.LARGE)}")
print(f"Parked bike spots: {parking_lot.parked_bike_spots()}")
print(f"Parked car spots: {parking_lot.parked_car_spots()}")
print(f"Parked truck spots: {parking_lot.parked_truck_spots()}")


Total spots: 35
Remaining spots: 35
Is parking lot full? False
Is parking lot empty? True
Available motorcycle spots: 10
Available car spots: 20
Available truck spots: 5
Parked bike spots: 0
Parked car spots: 0
Parked truck spots: 0
Parked truck at spot SpotType.LARGE
Parked truck at spot SpotType.LARGE
Parked truck at spot SpotType.LARGE
Parked truck at spot SpotType.LARGE
Parked truck at spot SpotType.LARGE
Parked truck at spot SpotType.MEDIUM
Parked truck at spot SpotType.MEDIUM
Parked truck at spot SpotType.MEDIUM
Parked truck at spot SpotType.MEDIUM
Parked truck at spot SpotType.MEDIUM
Parked truck at spot SpotType.MEDIUM
Total spots: 35
Remaining spots: 12
Is parking lot full? False
Is parking lot empty? False
Available small spots: 10
Available medium spots: 2
Available large spots: 0
Parked bike spots: 0
Parked car spots: 0
Parked truck spots: 11
